In [3]:
import json
CO_NUMS = list(range(1, 18))
EX_NUMS = list(range(1, 15))
co_datas = []
ex_datas = []
for name in CO_NUMS:
    with open('./CO/' + str(name) + '/data.json', 'r') as json_file:
        data = json.load(json_file)
    co_datas.append(data)
for name in EX_NUMS:
    with open('./EX/' + str(name) + '/data.json', 'r') as json_file:
        data = json.load(json_file)
    ex_datas.append(data)


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\GraduteThesis_Master\\EXP_DATA\\CO\\1\\data.json'

In [2]:
import numpy as np
co_familiar = []
ex_familiar = []
for item in co_datas:
    co_familiar.append(item['post'][0])
for item in ex_datas:
    ex_familiar.append(item['post'][0])
print(np.mean(co_familiar))
print(np.mean(ex_familiar))

2.5294117647058822
3.7142857142857144


In [3]:
import numpy as np
from scipy.stats import levene, bartlett

# 假设你有多个样本组，存储在 sample1, sample2, sample3 等数组中

# 进行方差齐性检验
# 使用 Levene's test
levene_test_statistic, levene_p_value = levene(co_familiar, ex_familiar)

# 使用 Bartlett's test
bartlett_test_statistic, bartlett_p_value = bartlett(co_familiar, ex_familiar)

# 打印检验结果
print("Levene's Test:")
print(f"Test Statistic: {levene_test_statistic}")
print(f"P-value: {levene_p_value}")
print()

print("Bartlett's Test:")
print(f"Test Statistic: {bartlett_test_statistic}")
print(f"P-value: {bartlett_p_value}")

Levene's Test:
Test Statistic: 0.8368213243441477
P-value: 0.3678514256501815

Bartlett's Test:
Test Statistic: 0.7341150610838801
P-value: 0.3915525803155623


In [4]:
from scipy.stats import mannwhitneyu

# 假设你有两个独立的样本组，存储在 sample1 和 sample2 中

# 执行 Mann-Whitney U 检验
statistic, p_value = mannwhitneyu(co_familiar, ex_familiar, alternative='two-sided')

# 打印检验结果
print("Mann-Whitney U Test:")
print(f"Test Statistic: {statistic}")
print(f"P-value: {p_value}")

# 根据 P 值判断是否存在显著差异
alpha = 0.05  # 显著性水平
if p_value < alpha:
    print("存在显著差异")
else:
    print("不存在显著差异")

Mann-Whitney U Test:
Test Statistic: 78.5
P-value: 0.10437023619972746
不存在显著差异
